The Objective of this notebook is to go through a list of weather stations and collect the weather data for each station. The data is collected from the Canada open data portal. Data for each station is combined into a single table then stored in a database for further analysis.

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import sqlalchemy as sq
import ipyparallel as ipp
from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from DataService import DataService

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
os.chdir('/tf')


2022-12-15 03:50:05.755257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 03:50:05.882997: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')

In [3]:
dataService = DataService(PGDB, PGUSER, PGPW)
db_con = dataService.connect()

In [4]:
tableName = "public.\"TenYrStationsHourly\""
query = "SELECT * FROM " + tableName + ";"
dfStations = gpd.GeoDataFrame.from_postgis(query, db_con)


In [5]:
def push_data(df: pd.DataFrame, tableName: str) -> None:
    df.to_sql(tableName, db_con, if_exists="append", index=False)


In [6]:
def validateColumnNames(curr: list, exp: list) -> bool:
    for name in curr:
        if name not in exp:
            return False
    return True

In [7]:
def dataProcessHourly(df: pd.DataFrame) -> None:
    NULLFLAG = -9999
    try:
        # discard	discard	discard	keep	discard	discard	keep	keep	keep	keep	keep	keep	discard	keep	discard	discard	discard	keep	discard	keep	discard	keep	discard	discard	discard	keep	discard	keep	discard	keep	discard	
        # x	y	STATION_NAME	CLIMATE_IDENTIFIER	ID	LOCAL_DATE	PROVINCE_CODE	LOCAL_YEAR	LOCAL_MONTH	LOCAL_DAY	LOCAL_HOUR	TEMP	TEMP_FLAG	DEW_POINT_TEMP	DEW_POINT_TEMP_FLAG	HUMIDEX	HUMIDEX_FLAG	PRECIP_AMOUNT	PRECIP_AMOUNT_FLAG	RELATIVE_HUMIDITY	RELATIVE_HUMIDITY_FLAG	STATION_PRESSURE	STATION_PRESSURE_FLAG	VISIBILITY	VISIBILITY_FLAG	WINDCHILL	WINDCHILL_FLAG	WIND_DIRECTION	WIND_DIRECTION_FLAG	WIND_SPEED	WIND_SPEED_FLAG	

        df.drop(columns=['x', 'y', 'STATION_NAME', 'ID', 'LOCAL_DATE', 'TEMP_FLAG', 'DEW_POINT_TEMP_FLAG', 'HUMIDEX', 'HUMIDEX_FLAG', 'PRECIP_AMOUNT_FLAG', 'RELATIVE_HUMIDITY_FLAG', 'STATION_PRESSURE_FLAG', 'VISIBILITY', 'VISIBILITY_FLAG', 'WINDCHILL_FLAG', 'WIND_DIRECTION_FLAG', 'WIND_SPEED_FLAG'], inplace=True)
    except:
        df.to_csv("Failed/" + str(df.iloc[0, 0]) +
                  "_unexpected_column_names.csv", index=False)

    expList = ['CLIMATE_IDENTIFIER', 'PROVINCE_CODE', 'LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY', 'LOCAL_HOUR', 'TEMP', 'DEW_POINT_TEMP', 'PRECIP_AMOUNT', 'RELATIVE_HUMIDITY', 'STATION_PRESSURE', 'WINDCHILL', 'WIND_DIRECTION', 'WIND_SPEED']
    currList = list(df.columns.values)
    
    if validateColumnNames(currList, expList):
        df.rename(columns={df.columns[0]: "ClimateID"}, inplace=True)
        df.rename(columns={df.columns[1]: "ProvinceCode"}, inplace=True)
        df.rename(columns={df.columns[2]: "Year"}, inplace=True)
        df.rename(columns={df.columns[3]: "Month"}, inplace=True)
        df.rename(columns={df.columns[4]: "Day"}, inplace=True)
        df.rename(columns={df.columns[5]: "Hour"}, inplace=True)
        df.rename(columns={df.columns[6]: "Temp"}, inplace=True)
        df.rename(columns={df.columns[7]: "DewPointTemp"}, inplace=True)
        df.rename(columns={df.columns[8]: "PrecipAmount"}, inplace=True)
        df.rename(columns={df.columns[9]: "RelativeHumidity"}, inplace=True)
        df.rename(columns={df.columns[10]: "StationPressure"}, inplace=True)
        df.rename(columns={df.columns[11]: "WindChill"}, inplace=True)
        df.rename(columns={df.columns[12]: "WindDirection"}, inplace=True)
        df.rename(columns={df.columns[13]: "WindSpeed"}, inplace=True)

        # df.dropna(subset=['Temp'], inplace=True)
        df.loc[df['Temp'].isnull(), 'Temp'] = NULLFLAG
        df.loc[df['DewPointTemp'].isnull(), 'DewPointTemp'] = NULLFLAG
        df.loc[df['PrecipAmount'].isnull(), 'PrecipAmount'] = NULLFLAG
        df.loc[df['RelativeHumidity'].isnull(), 'RelativeHumidity'] = NULLFLAG
        df.loc[df['StationPressure'].isnull(), 'StationPressure'] = NULLFLAG
        df.loc[df['WindChill'].isnull(), 'WindChill'] = NULLFLAG
        df.loc[df['WindDirection'].isnull(), 'WindDirection'] = NULLFLAG
        df.loc[df['WindSpeed'].isnull(), 'WindSpeed'] = NULLFLAG

        df[['ClimateID', 'ProvinceCode']] = df[['ClimateID', 'ProvinceCode']].astype(str)
        df[['Year', 'Month', 'Day', 'Hour']] = df[['Year', 'Month', 'Day', 'Hour']].astype(int)
        df[['Temp', 'DewPointTemp', 'PrecipAmount', 'RelativeHumidity', 'StationPressure', 'WindChill', 'WindDirection', 'WindSpeed']] = df[['Temp', 
                    'DewPointTemp', 'PrecipAmount', 'RelativeHumidity', 'StationPressure', 'WindChill', 'WindDirection', 'WindSpeed']].astype(float)

        # we try a db push, but if it fails, we place the data in a csv file
        # try:
        push_data(df, "WeatherDataHourlyTwentyYear")
        # db_con.execute(
        #     "UPDATE public.\"TenYrStationsHourly\" SET \"dataAvailable\" = True WHERE \"ClimateID\" like CAST(\'{}\' AS TEXT);".format(stationID))
        # except:
        #     df.to_csv("Failed/" + str(df.iloc[0, 0]) +
        #             "_data_failed_dbpush.csv", index=False)
    else:
        df.to_csv("Failed/" + str(df.iloc[0, 0]) +
                  "_error_column_names.csv", index=False)


In [12]:
#list of links
links = ['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=10000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=20000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=40000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=50000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=60000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=70000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=80000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=90000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=100000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=110000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=120000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=130000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=150000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=160000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=170000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=180000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=190000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=200000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=210000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=220000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=230000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=240000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=250000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=260000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=270000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=280000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=290000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=300000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=310000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=320000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=330000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=340000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=350000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=360000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=370000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=380000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=390000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=400000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=410000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=420000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=430000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=440000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=450000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=460000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=406QLD0|4061629|4061620|4067655|4063605&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=470000']

In [14]:
links2 = ['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=10000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=20000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=40000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=50000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=60000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=70000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=80000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=90000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=100000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=110000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=120000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=130000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=150000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=160000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=170000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=180000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=190000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=200000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=210000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=220000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=230000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=240000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=250000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=260000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=270000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=280000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=290000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=300000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=310000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=320000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=330000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=340000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=350000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=360000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=370000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=380000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=390000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=400000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=410000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=420000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=430000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=440000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=450000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=460000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=470000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=480000',
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=490000']

In [15]:
links3 = ['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=10000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=20000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=40000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=50000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=60000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=70000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=80000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=90000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=100000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=110000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=120000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=130000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=150000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=160000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=170000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=180000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=190000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=200000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=210000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=220000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=230000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=240000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=250000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=260000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=270000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=280000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=290000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=300000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=310000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=320000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=330000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=340000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=350000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=360000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=370000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=380000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=390000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=400000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=410000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=420000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=430000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=440000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=450000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=460000']

In [16]:
links4 =['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=10000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=20000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=40000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=50000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=60000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=70000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=80000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=90000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=100000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=110000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=120000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=130000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=150000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=160000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=170000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=180000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=190000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=200000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=210000']

In [17]:
links5 = ['https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=0',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=10000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=20000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=30000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=40000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=50000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=60000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=70000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=80000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=90000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=100000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=110000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=120000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=130000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=140000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=150000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=160000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=170000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=180000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=190000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=200000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=210000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=220000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=230000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=240000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=250000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=260000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=270000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=280000',
    'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex=290000']

In [13]:


for link in links:
    df = pd.read_csv(link)
    dataProcessHourly(df)

In [18]:
all = [links2, links3, links4, links5]
for oneList in all:
    for oneLink in oneList:
        df = pd.read_csv(oneLink)
        dataProcessHourly(df)
        clear_output(wait=False)
        print("Completed " + oneLink[-20:])


In [8]:
idList ={ '406QLD0' , '4061629', '4061620', '4067655', '4063605', 
          '4060984', '4060983', '4060988','4060982','4064150','4064155','4064149',
          '3062697','3062700','3062696','3072662','3072659','3072655',
          '5061645','5061649','5061648','2202202','2202200','2202201','2202203'}
baseUrl = 'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2000-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=' 
midUrl = '&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=10000&startindex='
index = 0
offset = 10000

In [14]:
df = pd.read_csv(baseUrl + '406QLD0' + midUrl + str(10000))


In [9]:
for id in idList:
    try:
        currIndex = 0
        for i in range(200):
            df = pd.read_csv(baseUrl + id + midUrl + str(currIndex))
            dataProcessHourly(df)
            currIndex += offset
            print("Completed " + id + " " + str(currIndex))

    except:
        print("Station: {} last offset below: {}".format(id, currIndex))
        continue

Completed 3062697 10000
Completed 3062697 20000
Completed 3062697 30000
Completed 3062697 40000
Completed 3062697 50000
Completed 3062697 60000
Completed 3062697 70000
Completed 3062697 80000
Completed 3062697 90000
Completed 3062697 100000
Station: 3062697 last offset below: 100000
Completed 4060982 10000
Completed 4060982 20000
Completed 4060982 30000
Completed 4060982 40000
Completed 4060982 50000
Completed 4060982 60000
Completed 4060982 70000
Completed 4060982 80000
Completed 4060982 90000
Station: 4060982 last offset below: 90000
Completed 3072659 10000
Completed 3072659 20000
Completed 3072659 30000
Completed 3072659 40000
Completed 3072659 50000
Completed 3072659 60000
Completed 3072659 70000
Completed 3072659 80000
Completed 3072659 90000
Station: 3072659 last offset below: 90000
Completed 2202202 10000
Completed 2202202 20000
Completed 2202202 30000
Completed 2202202 40000
Completed 2202202 50000
Completed 2202202 60000
Completed 2202202 70000
Completed 2202202 80000
Complete

In [ ]:
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=4060984|4060983|4060988|4060982|4064150|4064155|4064149&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=2000000&startindex=0'
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=3062697|3062700|3062696|3072662|3072659|3072655&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=2000000&startindex=0'
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=5061645|5061649|5061648&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=2000000&startindex=0'
'https://api.weather.gc.ca/collections/climate-hourly/items?datetime=2009-01-01%2000:00:00/2022-10-01%2000:00:00&CLIMATE_IDENTIFIER=2202202|2202200|2202201|2202203&sortby=PROVINCE_CODE,CLIMATE_IDENTIFIER,LOCAL_DATE&f=csv&limit=2000000&startindex=0'
